In [1]:
import os
import pandas as pd

In [2]:
fastq_dir = "/u/scratch/r/rwolff/Poyet_fastq_files/"
rd = pd.read_pickle("metagenomic_data_files/Poyet_repeat_day_samples.pkl")
hosts = ["am","an","ae","ao"]

This step merges the fastq's that were collected on the same day from the same host

In [3]:
for host in hosts:
    print(host)
    for date in rd[host].values():
        for f in date:
            for ind in [1,2]:
                os.system(f"cat {fastq_dir}{f}_{ind}.fastq.gz >> {fastq_dir}{date[0]}c_{str(ind)}.fastq.gz")

am
an
ae
ao


In [7]:
df=pd.read_csv("metagenomic_data_files/Poyet_SRA_report.txt",index_col=0,sep="\t")
df["host"] = [s[0] for s in df["sample_alias"].str.split("-")]

df_mg = df.loc[df["scientific_name"] == "human gut metagenome"]
df_mg = df_mg.loc[df_mg["library_strategy"] == "WGS"]

df_mg = df_mg.loc[df_mg["host"].isin(hosts)]

df_mg.index = df_mg["run_accession"]

cd = pd.read_pickle("metagenomic_data_files/Poyet_collection_dates.pkl")

In [8]:
df_mg["collection_date"] = cd

Create new metadata document with merged samples

In [10]:
for host in hosts:
    for tobe_merged in rd[host].values():
        df_mg_merged = df_mg.loc[tobe_merged]
        df_mg.loc[f"{tobe_merged[0]}c"] = df_mg_merged.sum()

        df_mg.loc[f"{tobe_merged[0]}c","sample_accession"] = " ".join(df_mg.loc[tobe_merged,"sample_accession"].values)
        df_mg.loc[f"{tobe_merged[0]}c","run_accession"] = " ".join(df_mg.loc[tobe_merged,"run_accession"].values)
        df_mg.loc[f"{tobe_merged[0]}c","sample_alias"] = " ".join(df_mg.loc[tobe_merged,"sample_alias"].values)
        df_mg.loc[f"{tobe_merged[0]}c","sra_ftp"] = " ".join(df_mg.loc[tobe_merged,"sra_ftp"].values)
        df_mg.loc[f"{tobe_merged[0]}c","fastq_ftp"] = " ".join(df_mg.loc[tobe_merged,"fastq_ftp"].values)

        df_mg.loc[f"{tobe_merged[0]}c","tax_id"] = df_mg.loc[tobe_merged,"tax_id"].values[0]
        df_mg.loc[f"{tobe_merged[0]}c","scientific_name"] = df_mg.loc[tobe_merged,"scientific_name"].values[0]
        df_mg.loc[f"{tobe_merged[0]}c","library_strategy"] = df_mg.loc[tobe_merged,"library_strategy"].values[0]
        df_mg.loc[f"{tobe_merged[0]}c","host"] = df_mg.loc[tobe_merged,"host"].values[0]
        df_mg.loc[f"{tobe_merged[0]}c","collection_date"] = df_mg.loc[tobe_merged,"collection_date"].values[0]
        df_mg.loc[f"{tobe_merged[0]}c","experiment_accession"] = df_mg.loc[tobe_merged,"experiment_accession"].values[0]

        df_mg = df_mg.drop(tobe_merged)

In [11]:
df_mg["collection_date"] = pd.to_datetime(df_mg["collection_date"])

In [12]:
df_mg = df_mg.sort_values(["host","collection_date"])

In [13]:
df_mg.to_csv("metagenomic_data_files/Poyet_sample_metadata.csv",sep="\t")

In [14]:
pd.Series(df_mg.groupby("host").groups["ae"]).to_csv("metagenomic_data_files/ae_samples.txt",sep="\n",index=None,header=None)

In [15]:
pd.Series(df_mg.groupby("host").groups["am"]).to_csv("metagenomic_data_files/am_samples.txt",sep="\n",index=None,header=None)

In [16]:
pd.Series(df_mg.groupby("host").groups["ao"]).to_csv("metagenomic_data_files/ao_samples.txt",sep="\n",index=None,header=None)

In [17]:
pd.Series(df_mg.groupby("host").groups["an"]).to_csv("metagenomic_data_files/an_samples.txt",sep="\n",index=None,header=None)

In [21]:
df_mg.loc["SRR9224019c"]

sample_accession                                SAMN11950361 SAMN11950362
experiment_accession                                           SRX5994824
run_accession                                       SRR9224019 SRR9224022
tax_id                                                             408170
scientific_name                                      human gut metagenome
library_strategy                                                      WGS
read_count                                                       10906280
base_count                                                     2078765697
fastq_ftp               ftp.sra.ebi.ac.uk/vol1/fastq/SRR922/009/SRR922...
submitted_ftp                                                           0
submitted_format                                                        0
sra_ftp                 ftp.sra.ebi.ac.uk/vol1/srr/SRR922/009/SRR92240...
sample_alias                                        ao-0014_MG ao-0015_MG
host                                  